# Preparação da Camada Gold — Open Meteo
Consolidação dos dados limpos da camada Silver em uma estrutura analítica pronta para consumo por dashboards e ferramentas de BI.



In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip
from pyspark.sql.functions import col, avg, min, max, count


In [2]:
builder = SparkSession.builder \
    .appName("GoldConsolidationOpenMeteo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()



25/04/14 15:46:18 WARN Utils: Your hostname, obi-wan-kenote resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/14 15:46:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kenote_ubuntu/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kenote_ubuntu/.ivy2/cache
The jars for the packages stored in: /home/kenote_ubuntu/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c865f4c8-f9fd-4731-af32-6322e3e817c3;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.3.0 in central
	found io.delta#delta-storage;3.3.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 145ms :: artifacts dl 5ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.3.0 from central in [default]
	io.delta#delta-storage;3.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     

In [3]:
#Leitura da Camada Silver

# Caminho absoluto da camada Silver
silver_path = "/home/kenote_ubuntu/projetos/Airflow/data/silver/open_meteo"

# Lendo os dados tratados da Silver
df_silver = spark.read.format("delta").load(silver_path)

# Visualizando os dados
df_silver.show(5)


25/04/14 15:46:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+------------+----------+-----------------+-----------------+
|        city|      date|temperature_min_c|temperature_max_c|
+------------+----------+-----------------+-----------------+
|   São Paulo|2025-04-27|             15.8|             24.8|
|   São Paulo|2025-04-28|             17.0|             26.1|
|   São Paulo|2025-04-29|             18.0|             20.0|
|Buenos Aires|2025-04-14|             10.7|             21.4|
|Buenos Aires|2025-04-15|             14.5|             19.6|
+------------+----------+-----------------+-----------------+
only showing top 5 rows



In [4]:
#Agregações para consolidar

# Consolidando por cidade: média, mínimo, máximo de temperaturas
df_gold = df_silver.groupBy("city").agg(
    count("*").alias("days_count"),
    avg("temperature_min_c").alias("avg_temp_min_c"),
    min("temperature_min_c").alias("min_temp_min_c"),
    max("temperature_min_c").alias("max_temp_min_c"),
    avg("temperature_max_c").alias("avg_temp_max_c"),
    min("temperature_max_c").alias("min_temp_max_c"),
    max("temperature_max_c").alias("max_temp_max_c")
)

# Arredondando (opcional, apenas visual)
df_gold = df_gold.select(
    "city", "days_count",
    col("avg_temp_min_c").cast("decimal(5,2)"),
    col("min_temp_min_c").cast("double"),
    col("max_temp_min_c").cast("double"),
    col("avg_temp_max_c").cast("decimal(5,2)"),
    col("min_temp_max_c").cast("double"),
    col("max_temp_max_c").cast("double")
)




In [5]:
#Visualização rápida da gold

df_gold.show(truncate=False)
df_gold.printSchema()



+------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+
|city        |days_count|avg_temp_min_c|min_temp_min_c|max_temp_min_c|avg_temp_max_c|min_temp_max_c|max_temp_max_c|
+------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+
|Bogotá      |16        |11.44         |9.2           |13.3          |18.51         |13.2          |21.5          |
|Buenos Aires|16        |13.94         |8.5           |17.0          |20.41         |18.1          |23.2          |
|Cabedelo    |16        |25.94         |25.0          |26.8          |28.96         |28.2          |30.1          |
|João Pessoa |16        |23.61         |21.7          |25.6          |30.19         |29.1          |31.3          |
|São Paulo   |16        |16.61         |14.3          |18.5          |22.23         |17.0          |26.1          |
|Santiago    |16        |14.26         |9.5           |19.1          |25

In [6]:
#Salvando a camada Gold como Delta

# Caminho absoluto da camada Gold
gold_path = "/home/kenote_ubuntu/projetos/Airflow/data/gold/open_meteo"

# Salvando no formato Delta
df_gold.write.format("delta") \
    .mode("overwrite") \
    .save(gold_path)



In [7]:
#Verificando os dados salvos

# Lendo novamente para validar o conteúdo da gold
df_check = spark.read.format("delta").load(gold_path)

# Conferindo o resultado final salvo
df_check.show(truncate=False)



+------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+
|city        |days_count|avg_temp_min_c|min_temp_min_c|max_temp_min_c|avg_temp_max_c|min_temp_max_c|max_temp_max_c|
+------------+----------+--------------+--------------+--------------+--------------+--------------+--------------+
|Bogotá      |16        |11.44         |9.2           |13.3          |18.51         |13.2          |21.5          |
|Buenos Aires|16        |13.94         |8.5           |17.0          |20.41         |18.1          |23.2          |
|Cabedelo    |16        |25.94         |25.0          |26.8          |28.96         |28.2          |30.1          |
|João Pessoa |16        |23.61         |21.7          |25.6          |30.19         |29.1          |31.3          |
|São Paulo   |16        |16.61         |14.3          |18.5          |22.23         |17.0          |26.1          |
|Santiago    |16        |14.26         |9.5           |19.1          |25

Conclusao
🔍 Leitura da silver Confirma que os dados foram salvos corretamente ✅ Agrupamento por cidade 🚨 Média, mínimo, máximo de temperaturar 📊 Salvamento no Delta 📈 Visualização Comunicação clara para quem for usar os dados depois 🧼 Pronto pra consumo em dashboard